In [1]:
## Import Utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from datetime import datetime, date

In [2]:
## Load Data (answers to questions)
with open('sx-stackoverflow-a2q.txt') as f:
    a2q = f.readlines()

In [3]:
## Load Data (comments to questions)
with open('sx-stackoverflow-c2q.txt') as f:
    c2q = f.readlines()

In [4]:
## Load Data (comments to answers)
with open('sx-stackoverflow-c2a.txt') as f:
    c2a = f.readlines()

In [5]:
## Init Graph
G = nx.DiGraph()

In [ ]:
len(a2q)

In [6]:
## Merge Graph about 1 year
t_start = datetime(2009, 10, 1)
t_end = datetime(2010, 10, 1)
for line in a2q:
    line = line.replace('\n', '')
    line = line.split(' ') 
    t = datetime.fromtimestamp(int(line[2]))
    t_truncated = date(t.year,t.month, t.day)
    if t_start <= t < t_end :
        if G.has_edge(line[0], line[1]) :
            G[line[0]][line[1]]['weight'] += 0.6
            G[line[0]][line[1]]['timestamp'].append(t_truncated)
        else :
            G.add_edge(line[0], line[1], weight=0.6)
            G[line[0]][line[1]]['timestamp']=[t_truncated]

for line in c2q:
    line = line.replace('\n', '')
    line = line.split(' ') 
    t = datetime.fromtimestamp(int(line[2]))
    t_truncated = date(t.year,t.month, t.day)
    if t_start <= t < t_end :
        if G.has_edge(line[0], line[1]) :
            G[line[0]][line[1]]['weight'] += 0.3
            G[line[0]][line[1]]['timestamp'].append(t_truncated)
        else :
            G.add_edge(line[0], line[1], weight=0.3)
            G[line[0]][line[1]]['timestamp']=[t_truncated]

for line in c2a:
    line = line.replace('\n', '')
    line = line.split(' ') 
    t = datetime.fromtimestamp(int(line[2]))
    t_truncated = date(t.year,t.month, t.day)
    if t_start <= t < t_end :
        if G.has_edge(line[0], line[1]) :
            G[line[0]][line[1]]['weight'] += 0.1
            G[line[0]][line[1]]['timestamp'].append(t_truncated)
        else :
            G.add_edge(line[0], line[1], weight=0.1)
            G[line[0]][line[1]]['timestamp']=[t_truncated]

Dijkstra's shortest path algorithm works as follows:

> Let distance of start vertex from start vertex = *0*. 

> Let distance of all other vertices from start = *+ Inf*.

**Repeat**
1. Visit the unvisited vertex with the smallest known distance from the start vertex.
2. For the current vertex, examine its unvisited neighbours.
3. For the current vertex, calculate distance of each neighbour from start vertex.
4. If the calculated distance of a vertex is less than the known distance, update the shortest distance.
5. Update the previous vertex for each of the updated distances.
6. Add the current vertex to the ist of a visited vertices.

Until all vertices visited.

In [7]:
## SubGraph 1 day
time_start = date(2010, 1, 6)
time_end = date(2010, 1, 7)
subG = nx.DiGraph()
for edge in list(G.edges):
    if time_start <= G.edges[edge[0], edge[1]]['timestamp'][0] < time_end:
        subG.add_edge(edge[0], edge[1])
        subG[edge[0]][edge[1]]['weight'] = G.edges[edge[0], edge[1]]['weight']
        subG[edge[0]][edge[1]]['timestamp'] = G.edges[edge[0], edge[1]]['timestamp']


In [8]:
import math

In [9]:
def shortest_path(G, source, target):
    unvisited = list(nx.nodes(G)) ## all nodes
    
    dist = {} ## distances from each node to source
    previous = {} ## previous node
    dist[source] = 0 ## The distance of the source node is 0
    for node in list(nx.nodes(G)):
        dist[node] = math.inf
        previous[node] = None
    
    while unvisited:  
        dist_min = math.inf
        for node in unvisited:
            if dist[node] <= dist_min :
                dist_min = dist[node]
                u = node
        unvisited.remove(u) ## removing the node from the list of the unvisited nodes
        
        ## if the visited node is the target stop the visits
        if u == target:
            break
            
        ## Examine its neighbours
        for neighbor in G.neighbors(u):
            new_dist = dist[u] + G[u][neighbor]['weight']
            if dist[neighbor] > new_dist:
                ## update the shortest distance
                dist[neighbor] = new_dist
                ## update the previous node
                previous[neighbor] = u
                
    if previous[target] == None:
        return ('Not Connected',[])
    
    path = [target]
    while source not in path:
        path.append(previous[path[-1]])
    path.reverse()

    return dist[target], path # return the distance

In [10]:
def func3(G, users, source, target):
    new_users = [source] + users + [target]
    res_path = [source]
    Dist = 0
    
    for i in range(len(new_users) - 1):
        dist, path = shortest_path(G, new_users[i], new_users[i+1])
        if dist == 'Not Connected':
            return ('Not Connected')
        Dist += dist
        res_path += path[1:]
    
    return Dist, res_path

In [13]:
len(subG.nodes)

4361

In [18]:
## Test 01
func3(subG,['19068','83473'], '78182', '221149')

'Not Connected'

In [19]:
## Test 02
func3(subG,['51173','52463'], '102896', '244170')

(5.7,
 ['102896',
  '93468',
  '19635',
  '244170',
  '74022',
  '63756',
  '51173',
  '188822',
  '39321',
  '55774',
  '158109',
  '5380',
  '52463',
  '83109',
  '135152',
  '215752',
  '175611',
  '244170'])